### La función 'Importar_Limpiar_Transformar' importa toda la base excepto la columna 'Causa externa' la limpia y hace algunos cambios

In [1]:
def Importar_Limpiar_Transformar(base_excel):
    import pandas as pd
    import numpy as np
    
    datos = pd.read_excel(base_excel)
    
    # Remplazar inf con None
    datos_ = datos.replace([np.inf, -np.inf], None)
    
    # Renombrar algunas variables
    datos_ = datos_.rename(columns={'Valor neto': 'Ingreso', 'Mon.': 'Moneda', 
                      'Código Episodio':'Código Paciente', 'Creado el': 'Fecha',
                              'Centro de Responsabilidad': 'Centro'})
    
    # La columna 'Edad' tiene formato object y además sus valores incluye y han letras
    # Vamos a eliminar las letras y dejar solo valores numéricos
    datos_['Edad'] = datos_['Edad'].str.replace(r'\D', '', regex=True)

    # Convertir el tipo de datos a floats
    datos_['Edad'] = pd.to_numeric(datos_['Edad'], errors='coerce')

    # Eliminamos y comprobamos la dimensión final
    datos_ = datos_.dropna(subset=['Edad', 'Pais de Nacimiento'], how='any')
    
    # 'Edad', de float a int 
    # 'Centro' y 'Código Paciente', de int a object
    datos_ = datos_.astype({'Edad': 'int', 'Centro': 'object', 'Código Paciente': 'object'})
    
    # Eliminar duplicados y comprobar que no quede ninguno
    datos_ = datos_.drop_duplicates()
    
    # Eliminar "Causa externa"
    datos_ = datos_.drop('Causa Externa', axis=1)
    
    # Convertir valores de usd a cop
    datos_['Ingreso'] = np.where(datos_['Moneda'] == 'USD', 
                         datos_['Ingreso'] * 4000, datos_['Ingreso'])
    
    # Clasificar las edades en grupos
    datos_['Grupo de edad'] = None
    datos_['Grupo de edad'] = np.where((0 <= datos_['Edad'])
                                       & (datos_['Edad'] <= 12), 'Niño', datos_['Grupo de edad'])

    datos_['Grupo de edad'] = np.where((13 <= datos_['Edad'])
                                       & (datos_['Edad'] <= 19), 'Adolescente', datos_['Grupo de edad'])

    datos_['Grupo de edad'] = np.where((20 <= datos_['Edad'])
                                       & (datos_['Edad'] <= 39), 'Adulto Joven', datos_['Grupo de edad'])

    datos_['Grupo de edad'] = np.where((40 <= datos_['Edad'])
                                       & (datos_['Edad'] <= 59), 'Adulto', datos_['Grupo de edad'])

    datos_['Grupo de edad'] = np.where((60 <= datos_['Edad']), 'Adulto Mayor', datos_['Grupo de edad'])
    
    return datos_

### La función 'Importar_Fecha_e_Ingreso' importa solo las columnas 'Creado el' y 'Valor neto' y hace algunos cambio

In [2]:
def Importar_Fecha_e_Ingreso(base_excel):
    import pandas as pd
    import numpy as np
    
    datos = pd.read_excel(base_excel)
    
    # Renombrar algunas variables
    datos = datos.rename(columns={'Valor neto': 'Ingreso', 'Mon.': 'Moneda',
                                  'Creado el': 'Fecha'})
    
    # Eliminar duplicados y comprobar que no quede ninguno
    datos = datos.drop_duplicates()
    
    # Convertir valores de usd a cop
    datos['Ingreso'] = np.where(datos['Moneda'] == 'USD', 
                         datos['Ingreso'] * 4000, datos['Ingreso'])

    datos = datos[['Fecha', 'Ingreso']]
    
    # Remplazar inf con None
    datos = datos.replace([np.inf, -np.inf], None)
    
    return datos

### La función grouped_df crea la serie de tiempo final con una periodicidad semanal

In [3]:
def grouped_df(data):
    
    import pandas as pd
    import numpy as np
    
    df = pd.DataFrame()
    df['Ingreso'] = data['Ingreso']

    # Crear columnas 'Año', 'Mes' y 'Semana'
    df['Fecha'] = data['Fecha']
    df['Año'] = data['Fecha'].dt.year
    df['Mes'] = data['Fecha'].dt.month
    df['Semana'] = data['Fecha'].dt.isocalendar().week

    # Reiniciar el índice
    df = df.reset_index(drop=True)

    # Convertir Semana a tipo int
    df['Semana'] = df['Semana'].astype(int)

    # Si la semana corresponde a otro año cambio el año
    df['Año'] = np.where((df['Mes']==1) & (df['Semana']==52),
                                    df['Año'] - 1, df['Año'])

    df['Año'] = np.where((df['Mes']==12) & (df['Semana']==1),
                                    df['Año'] + 1, df['Año'])

    ################ Agrupar #################################
    lista_columnas = df.columns.tolist()
    lista_columnas.remove('Fecha')
    lista_columnas.remove('Año')
    lista_columnas.remove('Semana')
    lista_columnas.remove('Mes')

    df_grouped = df.groupby(['Año', 'Semana'])[lista_columnas].sum().reset_index()
    df_grouped = df_grouped.rename(columns={'Ingreso': 'Ingreso SEMANAL'})

    
    ###################### Crear una fecha a partir del año y la semana #################
    # La fecha se establece en el primer día de la semana (lunes)
    
    año = df_grouped.loc[0, 'Año']
    semana = df_grouped.loc[0, 'Semana']

    # Obtener la fecha correspondiente al primer día de la semana
    date = pd.to_datetime(f'{año}-W{semana}-1', format='%Y-W%W-%w')
    
    fecha_inicio = str(date)     # Fecha de inicio
    periodos = len(df_grouped)                                         # Número de períodos
    frecuencia = 'W'             # Frecuencia: 'D' para diario, 'W' para semanal, 'M' para mensual
    
    # Crear un rango de fechas
    rango_fechas = pd.date_range(start=fecha_inicio, periods=periodos, freq=frecuencia)

    # Crear un DataFrame con el rango de fechas
    df_grouped['Fecha'] = pd.DataFrame(rango_fechas, columns=['Fecha'])

    # Crear columna 'Mes'
    df_grouped['Mes'] = df_grouped['Fecha'].dt.month
    # Convertir los números de mes a nombres de mes abreviados
    df_grouped['Mes'] = pd.to_datetime(df_grouped['Mes'].astype(str) + '-01', 
                                       format='%m-%d').dt.strftime('%b')
    

    ########## Mover la columnas 'Fecha', 'Año', 'Mes' a la primera posición ############
    columna_a_mover = ['Fecha', 'Año', 'Mes']
    columnas = columna_a_mover + [col for col in df_grouped.columns if col not in columna_a_mover]
    df_grouped = df_grouped[columnas]
    
    return df_grouped

### La función pronóstico_arima aplica un modelo arima(1,1,1) y hace gráfico

In [4]:
def pronóstico_arima(ts, order, semanas):
    # Fit the ARIMA model
    import pandas as pd
    import matplotlib.pyplot as plt
    from statsmodels.tsa.arima.model import ARIMA

    dates = ts['Fecha']
    ts_data = pd.Series(ts['Ingreso SEMANAL'].tolist(), index=dates)
    
    model = ARIMA(ts_data, order=order, freq='W')  # Example order (p=1, d=1, q=1)
    model_fit = model.fit()
    print(model_fit.summary())
    print()

    # Pronosticar los siguientes semanas
    forecast = model_fit.get_forecast(steps=semanas)
    conf_int = forecast.conf_int()
    forecast_index = pd.date_range(start=ts_data.index[-1] + pd.Timedelta(weeks=1), 
                                   periods=semanas, freq='W-SUN')
    print(f'Pronóstico Ingreso de las próximas {semanas} semanas')
    print(forecast.predicted_mean)

    # Plot the original data and the forecast
    plt.figure(figsize=(10, 5), dpi=350)
    plt.plot(ts_data, label='Datos Originales')
    plt.plot(forecast_index, forecast.predicted_mean, label='Pronóstico', color='red')
    plt.fill_between(forecast_index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], 
                     color='pink', alpha=0.5, label='95% Confidence Interval')
    plt.title('Pronóstico ARIMA')
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()